# Interventional Data Fusion
Now that a model describing the prostate gland motion has been built before an intervention, it can be fitted to the data acquired during that intervention.
<p style="float: left; width: 75%; margin-right: 1%;"><img src="./media/Registration_mini.gif" /></p>

## Load saved SSM model:

In [ ]:
import numpy as np

smm = np.load('./data/my_smm.npz')
print(smm.files)

In [ ]:
mX = smm['arr_0']
PCs = smm['arr_1']
lambdas = smm['arr_2']
print(mX.shape)
print(PCs.shape)
print(lambdas.shape)

## Load the test data:
This represents the "intraoperative data" available, such as the boundary points found on the prostate gland on ultrasound images.

In [ ]:
nodes_test = np.load('./data/nodes_test.npy')
print(nodes_test.shape)

## DIY
Try to understand what the following code is doing:

In [ ]:
X = np.reshape(nodes_test,[642*3,100],order='F')
X = X - mX

In [ ]:
nPCs = 5
B = np.matmul(np.transpose(PCs[:,0:nPCs]), X)  # projection
X_recon = np.matmul(PCs[:,0:nPCs], B)  # reconstruction
diff = X_recon-X
print(PCs[:,0:nPCs].shape)
print(np.sqrt(np.mean(np.square(diff))))

In [ ]:
nodes_recon = np.reshape(X_recon + mX, [642, 3, 100], order='F')

In [ ]:
tris = np.load('./data/tris.npy') 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
idx_shape = 10
ax.plot_trisurf(nodes_recon[:,0,idx_shape], 
                nodes_recon[:,1,idx_shape], 
                nodes_recon[:,2,idx_shape], 
                triangles=tris, 
                cmap=plt.cm.gist_heat)

## Noisy and missing data
A more interesting and realistic scenario is that the full segmentation of gland surface (all the points in X) is not available, all surgeon has during the procedure are some sparse points on some of the ultrasound slices and, potentially, with manual delineation errors.

Use the following lines or similar to take out some of the points data and/or add noise to the point coordinates:

In [ ]:
purge_rate = 0.5
noise_level = 0.1

num_valid = np.round(642*(1-purge_rate)).astype('int')
idx_valid = np.random.choice(np.arange(642), size=num_valid, replace=False)
# idx_valid = np.random.randint(0, 642, num_valid)
X = np.reshape(nodes_test[idx_valid,:,:],[num_valid*3,100],order='F')
X = X + np.random.normal(0,noise_level,X.shape)

print(num_valid)
print(X.shape)

Now experiment with the "corrupted" data, see how much the model still functions with increasing level of noise and fewer data points. 
Hint: 
- What means "being functional" needs to be quantified;
- The missing data do not have the "right size" for matrix multiplication any longer;
- Visualising your fitted model and data usually helps

footnote: this is a simplified application that, for instance, assumes known correspondence, in which you know which test point should align with a particular point in the model, an assumption does not hold in real-world application with many potential solutions such as iterative closest point algorithm or [ICP][icp].

[icp]: https://en.wikipedia.org/wiki/Iterative_closest_point